In [291]:
import pandas as pd

df = pd.read_csv("file.csv")

df["Implied Volatility"] = df["Implied Volatility"].str.replace('%', '').astype(float)
df = df[df["Implied Volatility"] > 0]

In [292]:
import yfinance as yf
from datetime import datetime, timedelta


date_end     = datetime.today()
date_start   = date_end - timedelta(weeks=1)

spx = yf.download('^GSPC', interval='1m', start=date_start, end=date_end)

[*********************100%***********************]  1 of 1 completed


In [293]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq

def black76_price(F, K, T, r, sigma, option_type='call'):
    """
    Black-76 formula for pricing European options on futures.
    
    Parameters:
    F : float       # Forward price (for SPX, can use spot as proxy if dividend = 0)
    K : float       # Strike price
    T : float       # Time to maturity (in years)
    r : float       # Risk-free interest rate
    sigma : float   # Implied volatility
    option_type : str, 'call' or 'put'
    
    Returns:
    float : Option price
    """
    d1 = (np.log(F / K) + 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    df = np.exp(-r * T)  # Discount factor

    if option_type == 'call':
        price = df * (F * norm.cdf(d1) - K * norm.cdf(d2))
    else:
        price = df * (K * norm.cdf(-d2) - F * norm.cdf(-d1))

    return price

In [294]:

def get_price(date_str):
    # Step 1: Parse input as naive datetime
    naive_date = datetime.strptime(date_str, '%m/%d/%Y %I:%M %p')

    # Step 2: Localize to US/Eastern (market timezone), then convert to UTC
    local_tz = pytz.timezone('US/Eastern')
    local_date = local_tz.localize(naive_date)
    date_utc = local_date.astimezone(pytz.utc)

    # Step 5: Try to get the exact timestamp, or handle missing
    try:
        S0 = spx.loc[date_utc]["Close"]
        return float(S0)
    except KeyError:
        return None  # Or optionally: find closest time, interpolate, etc.
    

df["S0"] = df['Last Trade Date (EDT)'].apply(get_price)

df.dropna(subset=["S0"], inplace=True)


exipry_date  = '2025-11-21'

df["T"] = (pd.to_datetime(exipry_date) - pd.to_datetime(df['Last Trade Date (EDT)'])).dt.days / 365.0

df["Last Price"] = df["Last Price"].replace(',', '', regex=True).astype(float)

df["Bid"] = df["Bid"].replace(',', '', regex=True).astype(float)
df["Ask"] = df["Ask"].replace(',', '', regex=True).astype(float)

df.head(10)

C:\Users\santo\AppData\Local\Temp\ipykernel_1760\3529378150.py:13: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(S0)


,Contract Name,Last Trade Date (EDT),Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility,S0,T
13,SPX251121C04000000,6/3/2025 10:09 AM,4000,2014.45,2050.2,2055.8,0.00,0.00%,5,153,47.49,5937.000000,0.465753
30,SPX251121C05000000,6/4/2025 12:02 PM,5000,1117.45,1115.2,1119.2,34.75,3.21%,1,98,32.51,5977.870117,0.463014
43,SPX251121C05300000,6/4/2025 11:26 AM,5300,859.55,853.5,857.2,9.57,1.13%,4,280,28.74,5982.609863,0.463014
44,SPX251121C05310000,5/30/2025 3:29 PM,5310,802.21,843.9,849.0,0.00,0.00%,1,0,28.64,5917.350098,0.476712
45,SPX251121C05320000,6/2/2025 3:28 PM,5320,805.85,836.8,841.4,0.00,0.00%,1,0,28.59,5934.509766,0.468493
48,SPX251121C05375000,5/29/2025 2:19 PM,5375,733.17,791.2,793.2,0.00,0.00%,1,116,27.77,5892.939941,0.479452
49,SPX251121C05400000,6/4/2025 11:26 AM,5400,776.02,770.6,772.7,9.67,1.26%,5,249,27.50,5982.609863,0.463014
51,SPX251121C05450000,6/3/2025 1:53 PM,5450,729.72,729.4,731.7,0.00,0.00%,1,561,26.92,5976.100098,0.465753
53,SPX251121C05500000,6/4/2025 11:20 AM,5500,694.68,688.2,690.3,41.17,6.30%,1,407,26.28,5984.870117,0.463014
54,SPX251121C05525000,5/30/2025 3:01 PM,5525,615.87,667.9,670.1,0.00,0.00%,29,179,25.99,5900.209961,0.476712


In [299]:
import numpy as np
from scipy.optimize import newton
from scipy.stats import norm

def black76_price(F, K, T, r, sigma, option_type='call'):
    """
    Black-76 option pricing model.
    F: forward price
    K: strike price
    T: time to maturity
    r: risk-free rate
    sigma: volatility
    option_type: 'call' or 'put'
    """
    if T <= 0 or sigma <= 0:
        return max(0.0, (F - K) if option_type == 'call' else (K - F))

    d1 = (np.log(F / K) + 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = np.exp(-r * T) * (F * norm.cdf(d1) - K * norm.cdf(d2))
    elif option_type == 'put':
        price = np.exp(-r * T) * (K * norm.cdf(-d2) - F * norm.cdf(-d1))
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    
    return price

def implied_vol76(S, K, T, market_price, option_type='call'):
    """
    Calculate implied volatility using the Black-76 model.
    """
    r = 0.05

    def f(sigma):
        F = np.exp(r * T) * S  # Forward price from spot
        return black76_price(F, K, T, r, sigma, option_type) - market_price

    try:
        iv = newton(f, x0=1, tol=1e-8, maxiter=100)
        return iv
    except Exception as e:
        print(f"Implied vol error for S={S}, K={K}, T={T}, Mkt={market_price}: {e}")
        return np.nan

df['IV (BS-76)'] = df.apply(
    lambda row: implied_vol76(
        row["S0"],
        row["Strike"],
        row["T"],
        # row["Last Price"],  # Use mid price as market price
        (row["Bid"] + row["Ask"])/2,
        option_type='call'  # Replace with row["Type"] if your data has that
    ), axis=1
)



In [ ]:
import numpy as np
from scipy.optimize import newton
from scipy.stats import norm

def black73_price(S, K, T, r, sigma, option_type='call'):
    """
    Black-Scholes (1973) option pricing model for European options.
    S: spot price
    K: strike price
    T: time to maturity (in years)
    r: risk-free rate
    sigma: volatility
    option_type: 'call' or 'put'
    """
    if T <= 0 or sigma <= 0:
        return max(0.0, (S - K) if option_type == 'call' else (K - S))

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    
    return price

def implied_vol73(S, K, T, market_price, option_type='call'):
    """
    Calculate implied volatility using the Black-76 model.
    """
    r = 0.03

    def f(sigma):
        F = S
        return black73_price(F, K, T, r, sigma, option_type) - market_price

    try:
        iv = newton(f, x0=1, tol=1e-8, maxiter=100)
        return iv
    except Exception as e:
        print(f"Implied vol error for S={S}, K={K}, T={T}, Mkt={market_price}: {e}")
        return np.nan

df['IV (BS-73)'] = df.apply(
    lambda row: implied_vol73(
        row["S0"],
        row["Strike"],
        row["T"],
        # row["Last Price"],  # Use mid price as market price
        (row["Bid"] + row["Ask"])/2,
        option_type='call'  # Replace with row["Type"] if your data has that
    ), axis=1
)

df[["S0", "Strike", "T", "Implied Volatility", 'IV (BS-73)', 'IV (BS-76)']].head(50)


,S0,Strike,T,Implied Volatility,IV (BS-73),IV (BS-76)
13,5937.000000,4000,0.465753,47.49,0.445765,0.367984
30,5977.870117,5000,0.463014,32.51,0.263740,0.205037
43,5982.609863,5300,0.463014,28.74,0.235479,0.194499
44,5917.350098,5310,0.476712,28.64,0.273986,0.243071
45,5934.509766,5320,0.468493,28.59,0.265964,0.234004
48,5892.939941,5375,0.479452,27.77,0.279650,0.252089
49,5982.609863,5400,0.463014,27.50,0.227892,0.191760
51,5976.100098,5450,0.465753,26.92,0.227323,0.194151
53,5984.870117,5500,0.463014,26.28,0.218195,0.185755
54,5900.209961,5525,0.476712,25.99,0.257794,0.232811
